## SparkStreaming Hackathon
### Course: Real-time Data Analysis
### Authors: Ruben Tak, Nils Jennissen, David Landeo
This task involves setting up a data streaming pipeline to extract and process posts and comments from Reddit. The data will be structured and sent through a socket, then received and processed by another process. References to users, posts, and external sites will be extracted and counted, and the top 10 important words will be identified using TF-IDF. Optional features include sentiment analysis, additional metrics, saving results to a database, creating a Jupyter Notebook dashboard, and visualizing the results on a web page. The deliverables include Python code, instructions, output data files, and optional Docker setup.

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, count, window, lit, desc, split, explode
from pyspark.sql.types import ArrayType

import json
import re

# Function to extract references to users, posts, and external sites from text
def get_references(text):
    ''' This function extracts references to users, posts, and external sites from text.
        It returns a list of references.'''
    user_pattern = r'/u/(\w+)'
    post_pattern = r'/r/(\w+)'
    url_pattern = r'(https?://[^\s]+)'

    references = []

    # Search for user references
    match = re.search(user_pattern, text)
    while match:
        references.append(match.group(1))
        text = text[:match.start()] + text[match.end():]
        match = re.search(user_pattern, text)

    # Search for post references
    match = re.search(post_pattern, text)
    while match:
        references.append(match.group(1))
        text = text[:match.start()] + text[match.end():]
        match = re.search(post_pattern, text)

    # Search for url references
    match = re.search(url_pattern, text)
    while match:
        references.append(match.group(1))
        text = text[:match.start()] + text[match.end():]
        match = re.search(url_pattern, text)

    return references

# Create a SparkSession and StreamingContext
spark_conf = SparkConf().setAppName("reddit")
spark_session = SparkSession.builder.config(conf=spark_conf).getOrCreate()
streaming_context = StreamingContext(spark_session.sparkContext, 5)

# Create a DStream
lines = streaming_context.socketTextStream("localhost", 9999)
comments = lines.map(lambda json_data: json.loads(json_data))

# Define the schema for the DataFrame
schema = StructType([
    StructField("comment", StringType(), True),
    StructField("prev_comment", StringType(), True),
    StructField("post", StringType(), True),
    StructField("author", StringType(), True),
    StructField("link_url", StringType(), True),
    StructField("link_permalink", StringType(), True),
    StructField("post_date", StringType(), True),
    StructField("ups", StringType(), True),
    StructField("likes", StringType(), True),
    StructField("post_img", StringType(), True)
])

# Update the base_path according to your desired output location
base_path = "./data/raw/reddit_v5"

# Convert each RDD in the DStream to a DataFrame and process it
def process_rdd(time, rdd):
    ''' This function is applied to each RDD in the DStream.
        It converts the RDD to a DataFrame, extracts references to users, posts, and external sites,
        counts the occurrences of references in 60-second windows every 5 seconds,
        and saves the processed data to disk.'''
    if not rdd.isEmpty():
        df = spark_session.createDataFrame(rdd, schema)

        # Extract references to users, posts, and external sites
        df_with_refs = df.withColumn("refs", split(col("comment"), r'/u/(\w+)|/r/(\w+)|(https?://[^\s]+)'))

        # Count occurrences of references in 60-second windows every 5 seconds
        windowed_counts = df_with_refs \
            .withWatermark("post_date", "60 seconds") \
            .groupBy(window("post_date", "5 seconds")) \
            .agg(count("refs").alias("reference_count"))

        # Get top 10 words in window using word count
        word_count = df_with_refs.selectExpr("explode(split(comment, ' ')) as word") \
            .groupBy("word") \
            .agg(count("*").alias("count")) \
            .orderBy(desc("count")) \
            .limit(10)

        # Convert the top 10 words to a string representation
        top10_str = str(word_count.select("word").rdd.flatMap(lambda x: x).collect())

        # Add the top10 words to the dataframe
        df_with_refs = df_with_refs.withColumn("top10", lit(top10_str))

        # Get the time range of the data
        min_time = df.selectExpr("MIN(post_date)").first()[0]
        max_time = df.selectExpr("MAX(post_date)").first()[0]

        # Add the time, min_time, max_time to the dataframe
        df_with_refs = df_with_refs.withColumn("time", lit(time))
        df_with_refs = df_with_refs.withColumn("min_time", lit(min_time))
        df_with_refs = df_with_refs.withColumn("max_time", lit(max_time))

        # Save the processed data to disk with folder names separated by hyphens
        output_path = f"{base_path}/{time.strftime('%Y-%m-%d-%H-%M-%S')}"
        df_with_refs.write.json(output_path)

        # Print some information for verification
        # print(f"Time: {time}, Data Range: {min_time} - {max_time}")
        # Show the output
        df_with_refs.show()

comments.foreachRDD(process_rdd)

# Start the streaming context
streaming_context.start()
# No streaming_context.awaitTermination() added here to make the cell non-blocking and to use other cells in parallel.

/usr/local/spark/python/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, from_unixtime
from pyspark.sql.types import TimestampType
from pyspark.sql import functions as F
from pyspark.sql.window import Window

host = "localhost"
port = 9999

# Create a SparkSession
spark_conf = SparkConf().setAppName("reddit")
spark_session = SparkSession.builder.config(conf=spark_conf).getOrCreate()

# Update the input_path according to the location in the first code chunk
input_path = "./data/raw/reddit_v5/*/*.json"

schema = StructType([
    StructField("comment", StringType(), True),
    StructField("prev_comment", StringType(), True),
    StructField("post", StringType(), True),
    StructField("author", StringType(), True),
    StructField("link_url", StringType(), True),
    StructField("link_permalink", StringType(), True),
    StructField("post_date", StringType(), True),
    StructField("ups", StringType(), True),
    StructField("refs", StringType(), True),
    StructField("top10", StringType(), True),
    StructField("time", StringType(), True),
    StructField("min_time", StringType(), True),
    StructField("max_time", StringType(), True),
    StructField("average_sentiment", StringType(), True)
])

# Read the JSON data saved by the first code chunk
streaming_df = spark_session.readStream \
    .format("json") \
    .schema(schema) \
    .option("path", input_path) \
    .load()

# Perform transformations on the data
transformed_df = streaming_df \
    .withColumn('post_date2', col('post_date').cast("float")) \
    .withColumn('post_date3', col('post_date2').cast("int")) \
    .withColumn('post_date_ts', from_unixtime(col('post_date3')).cast(TimestampType())) \
    .withColumn('min_time2', col('min_time').cast("float")) \
    .withColumn('min_time3', col('min_time2').cast("int")) \
    .withColumn('min_time_ts', from_unixtime(col('min_time3')).cast(TimestampType())) \
    .withColumn('max_time2', col('max_time').cast("float")) \
    .withColumn('max_time3', col('max_time2').cast("int")) \
    .withColumn('max_time_ts', from_unixtime(col('max_time3')).cast(TimestampType())) \
    .drop('post_date', 'min_time', 'max_time', 'post_date2', 'post_date3', 'min_time2', 'min_time3', 'max_time2', 'max_time3')

# Save the transformed data to disk
# Update the output_path and checkpt_path each time you rerun this cell
output_path = "./data/processed/reddit_v1"
checkpt_path = "./metadata/processed/reddit_v1"

transformed_df.writeStream \
    .format("json") \
    .option("checkpointLocation", checkpt_path) \
    .option("path", output_path) \
    .outputMode("append") \
    .start()

+--------------------+--------------------+--------------------+---------+--------------------+--------------------+-------------+---+--------------------+--------------------+-------------------+-------------+-------------+-----------------+
|             comment|        prev_comment|                post|   author|            link_url|      link_permalink|    post_date|ups|                refs|               top10|               time|     min_time|     max_time|average_sentiment|
+--------------------+--------------------+--------------------+---------+--------------------+--------------------+-------------+---+--------------------+--------------------+-------------------+-------------+-------------+-----------------+
|You're a complete...|We can always do ...|Dear redditors,\n...|harley247|https://www.reddi...|https://www.reddi...|1.687317098E9|  1|[You're a complet...|['moron', 'dude',...|2023-06-21 19:45:55|1.687317098E9|1.687317098E9|             null|
+--------------------+------